# G2EGM vs. NEGM

**Table of contents**<a id='toc0_'></a>    
- 1. [Setup](#toc1_)    
- 2. [Choose number of threads in numba](#toc2_)    
- 3. [Settings](#toc3_)    
- 4. [Timing function](#toc4_)    
- 5. [NEGM](#toc5_)    
- 6. [G2EGM](#toc6_)    
- 7. [Table](#toc7_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

This notebook produces the timing and accuracy results for the comparison of **NEGM** and **G2EGM** in [A Guide to Solve Non-Convex Consumption-Saving Models](https://doi.org/10.1007/s10614-020-10045-x).

## 1. <a id='toc1_'></a>[Setup](#toc0_)

In [1]:
%load_ext autoreload
%autoreload 2

import time
import numpy as np
np.seterr(all='ignore') # ignoring all warnings

# load the G2EGMModel module
from G2EGMModel import G2EGMModelClass
import figs
import simulate

## 2. <a id='toc2_'></a>[Choose number of threads in numba](#toc0_)

In [2]:
import numba as nb
nb.set_num_threads(1)

## 3. <a id='toc3_'></a>[Settings](#toc0_)

In [3]:
T = 20
Neta = 16
var_eta = 0.1**2
do_print = False

## 4. <a id='toc4_'></a>[Timing function](#toc0_)

In [4]:
def timing(model,
           rep=1, # set to 5 in the paper
           do_print=True):
    
    name = model.name
    par = model.par
    
    time_best = np.inf
    for i in range(rep):
        
        model.solve()
        model.calculate_euler()
            
        tot_time = np.sum(model.par.time_work)
        if do_print:
            print(f'{i}: {tot_time:.2f} secs, euler: {np.nanmean(model.sim.euler):.3f}')
            
        if tot_time < time_best:
            time_best = tot_time
            model_best = model.copy('best')
            
    model_best.name = name
    return model_best            

## 5. <a id='toc5_'></a>[NEGM](#toc0_)

In [5]:
model_NEGM = G2EGMModelClass(name='NEGM',par={'solmethod':'NEGM','T':T,'do_print':do_print})
model_NEGM.precompile_numba()
model_NEGM = timing(model_NEGM)

0: 153.76 secs, euler: -5.367


In [6]:
model_NEGM_shocks = G2EGMModelClass(name='NEGM_shocks',par={'solmethod':'NEGM','T':T,'do_print':do_print,'Neta':Neta,'var_eta':var_eta})
model_NEGM_shocks.precompile_numba()
model_NEGM_shocks = timing(model_NEGM_shocks)

0: 170.36 secs, euler: -5.201


## 6. <a id='toc6_'></a>[G2EGM](#toc0_)

In [7]:
model_G2EGM = G2EGMModelClass(name='G2EGM',par={'solmethod':'G2EGM','T':T,'do_print':do_print})
model_G2EGM.precompile_numba()
model_G2EGM = timing(model_G2EGM)       

0: 83.04 secs, euler: -6.233


In [8]:
model_G2EGM_shocks = G2EGMModelClass(name='G2EGM_shocks',par={'solmethod':'G2EGM','T':T,'do_print':do_print,'Neta':Neta,'var_eta':var_eta})
model_G2EGM_shocks.precompile_numba()
model_G2EGM_shocks = timing(model_G2EGM_shocks)

0: 105.80 secs, euler: -5.758


## 7. <a id='toc7_'></a>[Table](#toc0_)

In [9]:
# a. models
models = [model_G2EGM,model_NEGM,model_G2EGM_shocks,model_NEGM_shocks]
postfix = '_G2EGM_vs_NEGM'

# b. euler erros
lines = []
txt = 'All (average)'
for i,model in enumerate(models):
    txt += f' & {np.nanmean(model.sim.euler):.3f}'
txt += '\\\\ \n'
lines.append(txt)

txt = '\\,\\,5th percentile'
for i,model in enumerate(models):
    txt += f' & {np.nanpercentile(model.sim.euler,5):.3f}'
txt += '\\\\ \n'    
lines.append(txt)

txt = '\\,\\,95th percentile'
for i,model in enumerate(models):
    txt += f' & {np.nanpercentile(model.sim.euler,95):.3f}'
txt += '\\\\ \n'   
lines.append(txt)

with open(f'tabs_euler_errors{postfix}.tex', 'w') as txtfile:
    txtfile.writelines(lines)
    
# c. timings
lines = []
txt = 'Total'
for model in models:
    txt += f' & {np.sum(model.par.time_work)/60:.2f}'
txt += '\\\\ \n'
lines.append(txt)

txt = 'Post-decision functions'
for model in models:
    txt += f' & {np.sum(model.par.time_w)/60:.2f}'
txt += '\\\\ \n'    
lines.append(txt)

txt = 'EGM-step'
for model in models:
    txt += f' & {np.sum(model.par.time_egm)/60:.2f}'
txt += '\\\\ \n' 
lines.append(txt)

txt = 'VFI-step'
for model in models:
    tot_time = np.sum(model.par.time_vfi)
    if tot_time == 0:
        txt += f' & '
    else:
        txt += f' & {tot_time/60:.2f}'
txt += '\\\\ \n' 
lines.append(txt)

with open(f'tabs_timings{postfix}.tex', 'w') as txtfile:
    txtfile.writelines(lines)